In [57]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3
from varname import nameof


In [58]:
# # old
# df_forecast = pd.read_excel('data/Forecast_11_to_14.xlsx', usecols=['date', 'pvpower_instant'])
# df_forecast['date'] = pd.to_datetime(df_forecast['date'], format='%Y-%m-%dT%H:%M+00:00')
# df_forecast = df_forecast.loc[(df_forecast['date'].dt.day == 12) | (df_forecast['date'].dt.day == 13)]
# df_real = pd.read_excel('data/PV Generation-data-2022-11-01 12_52_19_neu.xlsx')
# df_real['Timestamp'] = pd.to_datetime(df_real['Timestamp'], format='%d.%m.%Y %H:%M:%S')

# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=df_real['Timestamp'], y=df_real['Heisenberg/71/260_READ_Power_DC1'], name='Real PV-Power'), secondary_y=True)
# fig.add_trace(go.Scatter(x=df_forecast['date'], y=df_forecast['pvpower_instant'], name='Meteoblue Forecast'), secondary_y=False)
# fig.update_layout(title_text='Vergleich MeteoBlue Forecast vs. Messdaten', title_x=0.5, template='plotly',
#                   xaxis_title='Zeit', xaxis=dict(tickformat='%d.%m %H:%M'),
#                   legend=dict(orientation='h', yanchor="top", y=0.99, xanchor="center", x=0.45))
# fig.update_yaxes(title_text='PV-Messwerte', secondary_y=True)
# fig.update_yaxes(title_text='MeteoBlue Forecast', secondary_y=False)
# fig.show()

In [59]:
#Check if connection is available
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()
#timerange = "Timestamp between '2022-07-19 00:00:00' and '2022-07-20 00:00:00' ORDER by Timestamp"
baseline=[pd.read_sql_query("SELECT * FROM baseline", conn).set_index('Timestamp'),'baseline']
household_batteries=[pd.read_sql_query("SELECT * FROM household_batteries ", conn).set_index('Timestamp'), 'household_batteries']
mb_basic=[pd.read_sql_query("SELECT * FROM mb_basic ", conn).set_index('Timestamp'),'mb_basic']
mb_pvpro_1h = [pd.read_sql_query("SELECT * FROM mb_pvpro_1h ", conn).set_index('Timestamp'), 'mb_pvpro_1h']
mb_clouds =[ pd.read_sql_query("SELECT * FROM mb_clouds ", conn).set_index('Timestamp'), 'mb_clouds']
mb_solar = [pd.read_sql_query("SELECT * FROM mb_solar ", conn).set_index('Timestamp'), 'mb_solar']
mb_sunmoon = [pd.read_sql_query("SELECT * FROM mb_sunmoon ", conn).set_index('Timestamp'), 'mb_sunmoon']
slp = [pd.read_sql_query("SELECT * FROM slp ", conn).set_index('Timestamp'), 'slp']
mb_pvpro_15min = [pd.read_sql_query("SELECT * FROM mb_pvpro_15min ", conn).set_index('Timestamp'), 'mb_pvpro_15min']
wunderground_historical_40 = [pd.read_sql_query("SELECT * FROM wunderground_historical_40 ", conn).set_index('Date'), 'wunderground_historical_40' ]
wunderground_historical_43 = [pd.read_sql_query("SELECT * FROM wunderground_historical_43 ", conn).set_index('Date'), 'wunderground_historical_43' ]
wunderground_historical_25 = [pd.read_sql_query("SELECT * FROM wunderground_historical_25 ", conn).set_index('Date'), 'wunderground_historical_25']

# baseline.name='baseline'
# household_batteries.name ='household_batteries'
# mb_basic.name = 'mb_basic'
# mb_pvpro_1h.name = 'mb_pvpro_1h'
# mb_clouds.name = 'mb_clouds'
# mb_solar.name = 'mb_solar'
# mb_sunmoon.name = 'mb_sunmoon'
# slp.name = 'slp'
# mb_pvpro_15min.name = 'mb_pvpro_15min'
# wunderground_historical_40.name = 'wunderground_historical_40' 
# wunderground_historical_43.name = 'wunderground_historical_43' 
# wunderground_historical_25.name = 'wunderground_historical_25'

df_list=[baseline, household_batteries, mb_basic, mb_pvpro_1h, mb_clouds, mb_solar, mb_sunmoon, slp, mb_pvpro_15min, wunderground_historical_40, wunderground_historical_43, wunderground_historical_25]
for df in df_list:
    df[0].index=pd.to_datetime(df[0].index)
    

In [60]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-10-01 00:00:00': '2022-10-02 00:00:00']     #hier Zeitraum auswählen; aber nicht zu groß, sonst schmiert das hier ab
    for c in tempDf.columns:
        if c in ['Baseline in kW' ]:
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c not in ['rainspot', 'visibility']:  
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.show()
#fig.show(renderer='browser')

C:\Users\lasse\AppData\Local\Temp\ipykernel_3140\1941989750.py:4: FutureWarning:

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.



Interessante Tage:

    Keine/wenig Verschattung:
        '2022-09-21 00:00:00'
        '2022-10-06 00:00:00'
        
    komplette Verschattung:
        '2022-09-24 00:00:00'
        '2022-04-04 00:00:00'

    teilweise bewölkt:
        '2022-10-17 00:00:00'
        '2022-10-16 00:00:00'
        '2022-10-08 00:00:00'

In [66]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-03-15 00:00:00': '2022-11-14 00:00:00']     #hier Zeitraum auswählen von bis
    for c in tempDf.columns:
        if c in ['Baseline in kW','solar_radiation', 'pvpower_instant']:             #hier Reihe für 1. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c in [ 'Pascal', 'Heisenberg', 'Einstein', 'Kelvin', 'Tesla']:                                   #hier Reihe für 2. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.show(renderer = 'browser')

C:\Users\lasse\AppData\Local\Temp\ipykernel_3140\2521266759.py:4: FutureWarning:

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.

